In [1]:
DATA_NAME = 'pastis-full' 
TRANSFORM = 'fourier'
CHANNEL = 'gray'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_gray_fourier_full_pastis.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10,11,...,24,25,26,27,28,29,30,31,32,33
2,12.61,-0.00,0.00,-0.00,-0.00,0.00,0.0,-0.00,0.00,-0.0,...,0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,0.0,0.0,-0.0
3,-0.00,9.73,0.00,0.00,0.00,0.00,-0.0,-0.00,0.00,-0.0,...,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.00,0.0,-0.0,-0.0
4,0.00,0.00,8.44,0.00,-0.00,-0.00,0.0,-0.00,0.00,-0.0,...,0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.0,0.0,0.0
5,-0.00,0.00,0.00,6.77,0.00,-0.00,0.0,0.00,0.00,0.0,...,0.00,0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.0,-0.0,0.0
6,-0.00,0.00,-0.00,0.00,5.57,0.00,0.0,-0.00,-0.00,-0.0,...,-0.00,0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.0,0.0,0.0
7,0.00,0.00,-0.00,-0.00,0.00,4.44,0.0,-0.00,0.00,0.0,...,-0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,-0.0,-0.0,0.0
8,0.00,-0.00,0.00,0.00,0.00,0.00,3.7,0.00,0.00,-0.0,...,-0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,0.0,-0.0,-0.0
9,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.0,3.08,-0.00,0.0,...,0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.0,0.0,-0.0
10,0.00,0.00,0.00,0.00,-0.00,0.00,0.0,-0.00,2.54,0.0,...,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.0,0.0,0.0
11,-0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.0,0.00,0.00,2.1,...,0.00,-0.00,0.00,-0.00,-0.00,0.00,-0.00,0.0,-0.0,-0.0


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10,11,...,24,25,26,27,28,29,30,31,32,33
2,1.00000,-0.00007,0.00002,-0.00003,-0.00002,0.00004,0.00003,-0.00004,0.00003,-0.00000,...,0.00004,0.00003,0.00004,-0.00005,-0.00000,-0.00000,0.00006,0.00004,0.00007,-0.00002
3,-0.00007,1.00000,0.00000,0.00002,0.00003,0.00005,-0.00002,-0.00003,0.00004,-0.00001,...,-0.00003,-0.00004,-0.00004,-0.00002,-0.00002,0.00000,0.00001,0.00003,-0.00002,-0.00006
4,0.00002,0.00000,1.00000,0.00007,-0.00003,-0.00001,0.00002,-0.00001,0.00002,-0.00000,...,0.00001,-0.00001,-0.00001,-0.00003,0.00003,-0.00003,-0.00000,0.00004,0.00001,0.00005
5,-0.00003,0.00002,0.00007,1.00000,0.00004,-0.00002,0.00002,0.00005,0.00001,0.00000,...,0.00002,0.00003,-0.00001,-0.00003,-0.00003,0.00003,-0.00000,0.00004,-0.00001,0.00001
6,-0.00002,0.00003,-0.00003,0.00004,1.00000,0.00000,0.00003,-0.00001,-0.00002,-0.00000,...,-0.00000,0.00001,-0.00000,-0.00002,0.00002,-0.00001,-0.00000,0.00002,0.00002,0.00001
7,0.00004,0.00005,-0.00001,-0.00002,0.00000,1.00000,0.00002,-0.00000,0.00001,0.00000,...,-0.00006,-0.00001,-0.00000,-0.00001,0.00004,0.00002,-0.00007,-0.00005,-0.00003,0.00008
8,0.00003,-0.00002,0.00002,0.00002,0.00003,0.00002,1.00000,0.00007,0.00004,-0.00000,...,-0.00005,0.00000,0.00002,0.00000,-0.00007,-0.00002,-0.00000,0.00001,-0.00004,-0.00003
9,-0.00004,-0.00003,-0.00001,0.00005,-0.00001,-0.00000,0.00007,1.00000,-0.00007,0.00001,...,0.00001,-0.00007,-0.00001,-0.00001,-0.00005,-0.00003,-0.00001,0.00001,0.00003,-0.00000
10,0.00003,0.00004,0.00002,0.00001,-0.00002,0.00001,0.00004,-0.00007,1.00000,0.00005,...,0.00001,-0.00001,0.00001,-0.00006,0.00002,-0.00001,0.00002,-0.00001,0.00001,0.00001
11,-0.00000,-0.00001,-0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00001,0.00005,1.00000,...,0.00002,-0.00003,0.00001,-0.00001,-0.00003,0.00002,-0.00005,0.00001,-0.00003,-0.00002


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.0021587542520741312

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[4.98732618e+00 2.96621011e+00 2.22433309e+00 1.43049929e+00
 9.66909085e-01 6.13623679e-01 4.26120108e-01 2.94330785e-01
 1.99521522e-01 1.35842843e-01 9.11176340e-02 6.66940051e-02
 3.99075366e-02 2.86735753e-02 1.87816825e-02 1.21799568e-02
 7.74551907e-03 4.87798607e-03 3.00744537e-03 1.92190512e-03
 1.11489675e-03 6.33218993e-04 3.51967474e-04 1.94408553e-04
 9.63389558e-05 4.48761473e-05 2.02586923e-05 8.25194979e-06
 1.18564286e-06 2.56165452e-07 4.29022018e-08 9.82815484e-31]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0.997129,-0.057639,-0.035807,-0.022075,-0.015983,-0.011607,-0.009271,-0.007533,-0.006046,-0.004938,...,-0.000247,-0.000186,-0.000131,-0.000095,-0.000063,-0.000042,-0.000014,-0.000007,-0.000003,-0.000002
1,0.051459,0.990948,-0.111413,-0.040007,-0.024888,-0.016675,-0.012926,-0.010215,-0.008066,-0.006530,...,-0.000329,-0.000247,-0.000176,-0.000120,-0.000082,-0.000054,-0.000020,-0.000010,-0.000005,-0.000003
2,0.038339,0.103897,0.989037,-0.080042,-0.040381,-0.024712,-0.018361,-0.014223,-0.011116,-0.008904,...,-0.000436,-0.000328,-0.000233,-0.000162,-0.000107,-0.000074,-0.000027,-0.000013,-0.000006,-0.000002
3,0.024268,0.042588,0.068752,0.990510,-0.092207,-0.039741,-0.026613,-0.019416,-0.014747,-0.011563,...,-0.000542,-0.000406,-0.000291,-0.000202,-0.000138,-0.000089,-0.000034,-0.000016,-0.000008,-0.000003
4,0.018819,0.028880,0.039337,0.081818,0.989853,-0.083108,-0.043725,-0.028946,-0.020769,-0.015765,...,-0.000704,-0.000526,-0.000375,-0.000260,-0.000175,-0.000117,-0.000044,-0.000021,-0.000009,-0.000005
5,0.013652,0.019472,0.024552,0.038384,0.070339,0.988261,-0.105870,-0.049138,-0.030643,-0.021771,...,-0.000886,-0.000657,-0.000467,-0.000322,-0.000215,-0.000144,-0.000058,-0.000028,-0.000012,-0.000005
6,0.011596,0.016074,0.019519,0.027866,0.041908,0.091783,0.985442,-0.111006,-0.050318,-0.031986,...,-0.001129,-0.000836,-0.000592,-0.000408,-0.000284,-0.000186,-0.000071,-0.000034,-0.000016,-0.000008
7,0.009915,0.013409,0.015980,0.021618,0.029897,0.049349,0.094538,0.984395,-0.111811,-0.052123,...,-0.001418,-0.001074,-0.000754,-0.000518,-0.000358,-0.000236,-0.000090,-0.000044,-0.000019,-0.000009
8,0.008262,0.011011,0.012999,0.017118,0.022510,0.032932,0.049228,0.093842,0.983453,-0.117239,...,-0.001785,-0.001338,-0.000947,-0.000659,-0.000442,-0.000296,-0.000111,-0.000055,-0.000024,-0.000011
9,0.007044,0.009307,0.010885,0.014063,0.017964,0.024775,0.033686,0.050468,0.098088,0.982237,...,-0.002259,-0.001701,-0.001198,-0.000829,-0.000566,-0.000372,-0.000144,-0.000069,-0.000032,-0.000015


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0028712703715808585,
 0.009051540542919234,
 0.010963274949280732,
 0.009490320717143508,
 0.010146626457403607,
 0.011738780353575784,
 0.014558428572623572,
 0.01560466242984726,
 0.016546795987715468,
 0.017762757920415484,
 0.02289007554529021,
 0.023199118037457556,
 0.0226147799561901,
 0.027412816078046687,
 0.025134068443891988,
 0.02591432838978658,
 0.027176293824519804,
 0.028932858441893305,
 0.03331588709921629,
 0.03571027997130749,
 0.033783990111921236,
 0.03618186136020196,
 0.040927312621779954,
 0.04368722924582802,
 0.04340594002376419,
 0.04880831151134002,
 0.058646179688227607,
 0.04736004038194097,
 0.032997802987156266,
 0.05819690266012478,
 0.1326472536636898,
 0.11575354866764542]